<a href="https://colab.research.google.com/github/tejasnikumbh999/Task_Guvi/blob/main/Task_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [146]:
f1=pd.read_csv('https://raw.githubusercontent.com/tejasnikumbh999/Task_Guvi/main/takehome_user_engagement.csv')

In [147]:
f2=pd.read_csv('https://raw.githubusercontent.com/tejasnikumbh999/Task_Guvi/main/takehome_users.csv',encoding='latin-1')

In [ ]:
f1.columns

In [ ]:
f2.columns

- Rename object_id by user_id to merge the two files

In [150]:
f2 = f2.rename(columns = {'object_id': 'user_id'}, inplace = False)

In [151]:
#merge the files

takehome = pd.merge(f1, f2, on='user_id')

In [ ]:
takehome.head()

#EDA

In [ ]:
#check null values

takehome.isnull().sum()

- The column invited_by_user_id has null values, thus delete the column.

In [62]:
takehome.drop(columns='invited_by_user_id',inplace=True)

In [ ]:
takehome.dtypes

In [154]:
#convert data types

takehome['creation_time']=pd.to_datetime(takehome['creation_time'])
takehome['time_stamp']=pd.to_datetime(takehome['time_stamp'])
takehome['last_session_creation_time']= pd.to_datetime(takehome['last_session_creation_time'],unit='s')

In [ ]:
#sort data weekly, monthly and yearly

takehome['Week_number'] = takehome['last_session_creation_time'].dt.week
takehome['week'] = takehome['last_session_creation_time'].dt.year.astype(str) + takehome['last_session_creation_time'].dt.week.astype(str)
takehome['year']=takehome['last_session_creation_time'].dt.year
takehome['month']=takehome['last_session_creation_time'].dt.month

In [ ]:
takehome.head()

- Week_number assigns the week for a particular year and may be same for same day on next year.
- week represents the cumulative week and is unique for each year.

In [203]:
#Group the data by week

Group = takehome.groupby(['user_id','org_id','creation_source','opted_in_to_mailing_list','enabled_for_marketing_drip','Week_number','week','month','year'])['visited'].count().reset_index()

In [204]:
#create the dataset of the users wher weekly visits are greater than 3

Group['adopted_user'] = Group['visited'].apply(lambda x: 'yes' if x >= 3 else 'no')

In [ ]:
Group

#Data Visualization

In [ ]:
fig = px.pie(Group, names='adopted_user', title='Adopted Users')
fig.show()

- Around 25% of the users are abopted i.e they are used to login more than three time in a week.

#Machine Learning

- Lable encoding

In [207]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [208]:
for column in Group.columns:
  if Group[column].dtype == object:
    Group[column] = le.fit_transform(Group[column])

In [ ]:
plt.figure(figsize=(20,5))
sns.heatmap(Group.corr(),annot=True)

In [277]:
from sklearn.model_selection import train_test_split
X=Group.drop(columns=['user_id','adopted_user','visited','year','Week_number'],axis=1)
Y=Group['adopted_user']
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=60)

- Random Forest

In [278]:
from sklearn.ensemble import RandomForestClassifier 
randomforest=RandomForestClassifier(max_depth=7,n_estimators=35,random_state=3)
randomforest.fit(x_train,y_train)
y_pred=randomforest.predict(x_test)
score=randomforest.score(x_test,y_test)
score*100

85.64412542500945

- XGBoost Algorithm

In [279]:
from xgboost import XGBClassifier
xgb = XGBClassifier(max_depth=2,n_estimators=100)
xgb.fit(x_train,y_train)
 
xgb_score = xgb.score(x_test,y_test)*100
print("XGBoost  Score is :",xgb_score)

XGBoost  Score is : 85.60634680770683


In [ ]:
#predict feature importance

feature_scores = pd.Series(randomforest.feature_importances_,index=x_train.columns).sort_values(ascending=False)
feature_scores = feature_scores*100
feature_scores


In [ ]:
#plot feature importance

plt.figure(figsize=(12,12))
sns.barplot(x=feature_scores,y=feature_scores.index)
plt.show()

- Thus the feature importance is as listed:

week >  month > org_id > creation_source > enabled_for_marketing_drip > opted_in_to_mailing_list	